<a href="https://colab.research.google.com/github/maddy147/reddit_aita/blob/main/senti_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install git+https://github.com/openai/CLIP.git
import clip
nltk.download('stopwords')
nltk.download('wordnet')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-qb7x7uq_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-qb7x7uq_
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/nlp/aita/data_1k.xlsx',index_col=0)

In [ ]:
valid_data = data.copy()
valid_data

,title,body,yta,nta
0,AITA for telling my wife the lock on my daught...,My brother in-law (Sammy) lost his home shortl...,0,123
1,META: This sub is moving towards a value syste...,I’ve enjoyed reading and posting on this sub f...,17,62
2,"UPDATE, AITA for despising my mentally handica...","I'm back like I said I would be,. My [original...",0,44
3,AITA For suing my girlfriend after she had my ...,I'll try to keep this short. I had a [1967 Imp...,0,156
4,UPDATE: AITA for wanting to go to the funeral ...,I want to sincerely thank everyone who comment...,0,0
...,...,...,...,...
993,AITA for leaving in the middle of celebration ...,\n\nI f25 married my husband m31 3 years ago. ...,0,148
994,Aita: For yelling at my fiance for giving my d...,"My best friend Kelly passed away 2 years ago, ...",1,204
995,AITA for labeling all the bottles in the house...,I have two of my children living with me. One ...,1,157
996,AITA for treating my two daughters' pregnancie...,"When my oldest told me she was pregnant, she w...",46,67


In [ ]:
valid_data = valid_data[~(valid_data['title'].str.contains('UPDATE'))]
valid_data = valid_data[~(valid_data['title'].str.contains('Update'))]
valid_data = valid_data[~(valid_data['title'].str.contains('META'))]
valid_data = valid_data[~(valid_data['title'].str.contains('meta'))]
valid_data = valid_data.reset_index()

In [ ]:
valid_data

,index,title,body,yta,nta
0,0,AITA for telling my wife the lock on my daught...,My brother in-law (Sammy) lost his home shortl...,0,123
1,3,AITA For suing my girlfriend after she had my ...,I'll try to keep this short. I had a [1967 Imp...,0,156
2,5,AITA for pretending to get fired when customer...,I am a high schooler with a weekend job at a c...,46,44
3,6,"AITA for bringing up my brother's ""premature"" ...",I am a nurse practitioner and I am the primary...,0,87
4,8,AITA for making a dad joke?,"Note. My step-daughter, Madeline, was about a ...",80,38
...,...,...,...,...,...
884,993,AITA for leaving in the middle of celebration ...,\n\nI f25 married my husband m31 3 years ago. ...,0,148
885,994,Aita: For yelling at my fiance for giving my d...,"My best friend Kelly passed away 2 years ago, ...",1,204
886,995,AITA for labeling all the bottles in the house...,I have two of my children living with me. One ...,1,157
887,996,AITA for treating my two daughters' pregnancie...,"When my oldest told me she was pregnant, she w...",46,67


In [ ]:
#object of WordNetLemmatizer
lm = WordNetLemmatizer()

In [ ]:
def text_transformation(df_col):
    corpus = []
    for item in df_col:
        new_item = re.sub('[^a-zA-Z]',' ',str(item))
        new_item = new_item.lower()
        new_item = new_item.split()
        new_item = [lm.lemmatize(word) for word in new_item if word not in set(stopwords.words('english'))]
        corpus.append(' '.join(str(x) for x in new_item))
    return corpus

In [ ]:
corpus = text_transformation(valid_data['body'])

In [ ]:
lbs = [int(y>n) for y,n in zip(valid_data['yta'],valid_data['nta'])]

In [ ]:
class dset(Dataset):
    def __init__(self,txt,lbs) -> None:
        super().__init__()
        self.txt = txt
        self.lbs = lbs
    def __len__(self):
        return len(self.lbs)
    def __getitem__(self, index):
        return self.txt[index],self.lbs[index]

In [ ]:
dataset = dset(corpus,lbs)

In [ ]:
traindata,testdata = train_test_split(dataset,random_state = 42)

In [ ]:
print(len(traindata))
print(len(testdata))

666
223


In [ ]:
cls = nn.Linear(512,2).to('cuda')
train_ldr = DataLoader(traindata,batch_size= 32)

In [ ]:
model, preprocess = clip.load("ViT-B/32", device='cuda')

In [ ]:
def run_epoch(train_ldr,model,cls,opt):
    ep_loss = 0
    for data in train_ldr:
        txt_features = torch.stack([model.encode_text(clip.tokenize(txt,truncate = True).cuda()) for txt in data[0]]).squeeze(1).float()
        loss = F.cross_entropy(cls(txt_features),data[1].to('cuda'))
        opt.zero_grad()
        loss.backward()
        opt.step()
        ep_loss += loss.item()

    ep_loss = ep_loss/len(train_ldr)

    return ep_loss


def train(train_ldr,model,cls,max_epochs):
    losses = []
    opt = torch.optim.SGD(list(model.parameters()) + list(cls.parameters()),lr=1e-4)
    for epoch in range(max_epochs):
        ep_loss =run_epoch(train_ldr,model,cls,opt)
        print("Loss at epoch {}:{}".format(epoch,ep_loss))
        losses.append(ep_loss)
    return losses

In [ ]:
losses = (train(train_ldr=train_ldr,model=model,cls=cls,max_epochs=100))
print(losses)

Loss at epoch 0:0.402486998410452
Loss at epoch 1:0.3094067055554617
Loss at epoch 2:0.28370718090307145
Loss at epoch 3:0.27077380745183854
Loss at epoch 4:0.26324939124640967
Loss at epoch 5:0.2584759873293695
Loss at epoch 6:0.2552427904946463
Loss at epoch 7:0.2529232143646195
Loss at epoch 8:0.2511684206270036
Loss at epoch 9:0.24975915856304623
Loss at epoch 10:0.24859885232789175
Loss at epoch 11:0.2475933102624757
Loss at epoch 12:0.24667156913450786
Loss at epoch 13:0.24585344074737459
Loss at epoch 14:0.24510537726538523
Loss at epoch 15:0.2444119325705937
Loss at epoch 16:0.24373887230952582
Loss at epoch 17:0.24311085293690363
Loss at epoch 18:0.24250313568682896
Loss at epoch 19:0.2419026139236632
Loss at epoch 20:0.2412984339254243
Loss at epoch 21:0.24070400567281813
Loss at epoch 22:0.24011280032850446
Loss at epoch 23:0.23949971156460897
Loss at epoch 24:0.23889356461309252
Loss at epoch 25:0.23824508168867656
Loss at epoch 26:0.23762473109222593
Loss at epoch 27:0.236

In [ ]:
torch.save(model,'/content/drive/MyDrive/nlp/models/body_as_text/fix_test_data/mod_100.pt')
torch.save(cls,'/content/drive/MyDrive/nlp/models/body_as_text/fix_test_data/cls_100.pt')
torch.save(losses,'/content/drive/MyDrive/nlp/models/body_as_text/fix_test_data/losses_100.pt')

In [ ]:
model = torch.load('/content/drive/MyDrive/nlp/models/body_as_text/fix_test_data/mod_100.pt')
cls = torch.load('/content/drive/MyDrive/nlp/models/body_as_text/fix_test_data/cls_100.pt')

In [ ]:
test_ldr =  DataLoader(testdata,batch_size= 32)
acc = 0
for d in test_ldr:
    logits = cls(torch.stack([model.encode_text(clip.tokenize(txt,truncate = True).cuda()) for txt in d[0]]).squeeze(1).float())
    pred = torch.argmax(logits,dim = 1)
    acc = acc + sum(pred == d[1].to("cuda"))/(32)

In [ ]:
acc/len(test_ldr)

tensor(0.9196, device='cuda:0')